In [50]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, RandomizedSearchCV, StratifiedKFold
from scipy.stats import expon, reciprocal, uniform
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, DotProduct, ExpSineSquared, RationalQuadratic
import numpy as np
from sklearn.feature_selection import RFE, SelectFromModel, RFECV
from sklearn.inspection import permutation_importance

In [38]:
def load_data(file_list, df_activities, df_links_network):
    data_frames = []
    for file in file_list:
        with open(file, 'r') as f:
            data = json.load(f)
            df_links = pd.DataFrame({
                'link_id': data['links_id'],
                'link_from': data['link_from'],
                'link_to': data['link_to'],
                'link_length': data['link_length'],
                'link_freespeed': data['link_freespeed'],
                'link_capacity': data['link_capacity'],
                'link_permlanes': data['link_permlanes'],
                'link_counts': data['link_counts']
            })
            df_nodes = pd.DataFrame({
                'node_id': data['nodes_id'],
                'node_x': data['nodes_x'],
                'node_y': data['nodes_y']
            })
            df_od_pairs = pd.DataFrame(data['o_d_pairs'], columns=['origin', 'destination'])
            
            df_work = pd.DataFrame({
                        'work_x': data['work_x'],
                        'work_y': data['work_y'],
                        'go_to_work': data['go_to_work']
            })
            df_home = pd.DataFrame({
                'home_x': data['home_x'],
                'home_y': data['home_y'],
                'go_to_home': data['go_to_home']
            })
            
            df_links = df_links.merge(df_nodes, how='left', left_on='link_from', right_on='node_id')
            df_links = df_links.rename(columns={'node_x': 'start_node_x', 'node_y': 'start_node_y'})
            df_links.drop('node_id', axis=1, inplace=True)
            df_links = df_links.merge(df_nodes, how='left', left_on='link_to', right_on='node_id')
            df_links = df_links.rename(columns={'node_x': 'end_node_x', 'node_y': 'end_node_y'})
            df_links.drop('node_id', axis=1, inplace=True) 
            
            origin_counts = df_od_pairs['origin'].value_counts()
            df_origin_counts = origin_counts.reset_index()
            df_origin_counts.columns = ['origin', 'start_count']
            destination_counts = df_od_pairs['destination'].value_counts()
            df_destination_counts = destination_counts.reset_index()
            df_destination_counts.columns = ['destination', 'end_count']
            df_links = df_links.merge(df_origin_counts, how='left', left_on='link_from', right_on='origin')
            df_links.drop('origin', axis=1, inplace=True)
            df_links = df_links.merge(df_destination_counts, how='left', left_on='link_to', right_on='destination')
            df_links.drop('destination', axis=1, inplace=True)
            df_links[['start_count','end_count']] = df_links[['start_count','end_count']].fillna(0)
            
            df_act_work = df_activities[df_activities['activity_type_main']=='work']
            df_act_work = df_act_work.merge(df_work, how='left', left_on=['x','y'], right_on=['work_x','work_y'])
            df_act_work.drop(['x','y'], axis=1, inplace=True)
            df_act_work_agg = df_act_work.groupby(by="link").sum()['go_to_work'].reset_index(drop=False)
            df_act_home = df_activities[df_activities['activity_type_main']=='home']
            df_act_home = df_act_home.merge(df_home, how='left', left_on=['x','y'], right_on=['home_x','home_y'])
            df_act_home.drop(['x','y'], axis=1, inplace=True)
            df_act_home_agg = df_act_home.groupby(by="link").sum()['go_to_home'].reset_index(drop=False)
            df_act_agg = df_act_home_agg.merge(df_act_work_agg, how='outer', on='link')
            df_act_agg.fillna(0, inplace=True)
            df_act_agg['go_to_sum'] = df_act_agg['go_to_home'] + df_act_agg['go_to_work']
            
            mg = df_links.merge(df_links_network, how='left', on=['start_node_x','start_node_y','end_node_x','end_node_y'])
            mg = mg[['link_id_x','link_from','link_to','link_id_y','from', 'to']]
            link_home_work = mg.merge(df_act_agg, how='left', left_on='link_id_y', right_on='link')
            link_home_work['go_to_sum'].fillna(0, inplace=True)
            link_go_to = link_home_work[['link_id_x', 'go_to_sum']]
            df_links = df_links.merge(link_go_to, how='left', left_on='link_id', right_on='link_id_x')
            df_links.drop('link_id_x', axis=1, inplace=True)
        data_frames.append(df_links)
    return pd.concat(data_frames, ignore_index=True)
# 
# 
train_files = ['s-0.json', 's-1.json', 's-2.json', 's-3.json', 's-4.json','s-5.json', 's-6.json', 's-7.json', 's-8.json', 's-9.json'] 
test_files = ['s-15.json', 's-16.json', 's-17.json', 's-18.json','s-19.json']
validate_files = ['s-10.json', 's-11.json', 's-12.json', 's-13.json','s-14.json']
train_files = ['Data/cutoutWorlds/Train/po-1_pn-1.0_sn-1/' + i for i in train_files]
test_files = ['Data/cutoutWorlds/Test/po-1_pn-1.0_sn-1/' + j for j in test_files]
validate_files = ['Data/cutoutWorlds/Validate/po-1_pn-1.0_sn-1/' + k for k in validate_files]
df_activities = pd.read_pickle("Data/cutoutWorlds/Train/po-1_pn-1.0_sn-1/df_activities.pkl")
df_links_network = pd.read_pickle("Data/cutoutWorlds/Train/po-1_pn-1.0_sn-1/df_links_network.pkl")
train_data = load_data(train_files, df_activities, df_links_network)
test_data = load_data(test_files, df_activities, df_links_network)
validate_data = load_data(validate_files, df_activities, df_links_network)


In [54]:
# # Initialize a list to hold trips
# trips = []
# current_trip = [df_od_pairs.iloc[0]['origin']]  # Start with the first origin
# 
# # Iterate over the DataFrame rows
# for i, row in df_od_pairs.iterrows():
#     current_trip.append(row['destination'])  # Always add the destination
#     # Check if the next origin matches the current destination
#     if i + 1 < len(df_od_pairs) and row['destination'] != df_od_pairs.iloc[i + 1]['origin']:
#         # If it doesn't, the current trip has ended
#         trips.append(current_trip)
#         current_trip = [df_od_pairs.iloc[i + 1]['origin']]  # Start a new trip
# 
# # Add the last trip if it wasn't already added
# if current_trip not in trips:
#     trips.append(current_trip)



In [55]:
# from collections import Counter
# # Flatten the list of trips into a single list of nodes including origins and destinations
# all_nodes = [node for trip in trips for node in trip]
# 
# # Use Counter to count the occurrences of each node
# node_trip_counts = Counter(all_nodes)
# 
# df_node_trip_counts = pd.DataFrame.from_dict(node_trip_counts, orient='index').reset_index()
# df_node_trip_counts.columns = ['node_id', 'trip_amount']

In [52]:
len(y_t)

16985

In [46]:
# 
numerical_features = ['link_length', 'link_freespeed', 'link_capacity', 'link_permlanes', 'start_node_x', 'start_node_y', 'end_node_x', 'end_node_y', 'start_count', 'end_count', 'go_to_sum']
X_t = train_data.drop(columns=['link_counts'])
y_t = train_data['link_counts']
X_v = validate_data.drop(columns=['link_counts'])
y_v = validate_data['link_counts']
X_te = test_data.drop(columns=['link_counts'])
y_te = test_data['link_counts']
scaler = StandardScaler()
X_t[numerical_features] = scaler.fit_transform(X_t[numerical_features])
X_v[numerical_features] = scaler.fit_transform(X_v[numerical_features])
X_te[numerical_features] = scaler.fit_transform(X_te[numerical_features])
# X_t = scaler.fit_transform(X_t)
# X_v = scaler.fit_transform(X_v)

In [51]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': LassoCV(cv=3, random_state=42, max_iter=100000),
    'Ridge': RidgeCV(cv=3),
    'SVR': SVR(C=25.383309585489613, epsilon=0.01, gamma=2.1969677491639246, max_iter=3000),
    'Random Forest': RandomForestRegressor(criterion='friedman_mse', max_depth=20, min_samples_leaf=2, n_estimators=150, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(max_depth=5, min_samples_split=5, random_state=42, subsample=0.8),
    'Artificial Neural Network': MLPRegressor(activation='tanh', alpha=0.001, learning_rate_init=0.01, max_iter=1000, random_state=42, solver='sgd'),
#     'Gaussian Process Regression': GaussianProcessRegressor(kernel=RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0), alpha=0.1, n_restarts_optimizer=3)
}

# Function to train and evaluate models
def evaluate_models(models, X_train, y_train, X_test, y_test):
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
#         kf = KFold(n_splits=3, shuffle=True, random_state=42)
#         cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
#         mse_scores = -cv_scores 
#         mean_mse = mse_scores.mean()
#         std_mse = mse_scores.std()
        # mape = mean_absolute_percentage_error(y_test, y_pred)
        # r2 = r2_score(y_test, y_pred)
        print(name + " done")
        
        results[name] = {'MAE': mae, 'MSE': mse}
    
    return results

# Function to train and evaluate models
def feature_select_models(models, X_train, y_train, X_test, y_test):
    results = {}
    for name, model in models.items():
        if name not in ['SVR', 'Artificial Neural Network', 'Gaussian Process Regression']:
            selector = RFECV(model, step=1, cv=StratifiedKFold(5), scoring='neg_mean_squared_error').fit(X_train, y_train)
            # To see which features are selected
            selected_features = X_train.columns[selector.support_]
            y_pred = selector.predict(X_test)
        else:
            # Fit Random Forest to get feature importances
            rf = RandomForestRegressor()
            rf.fit(X_train, y_train)

            # Select features based on importances
            rfe = RFECV(estimator=rf, step=1, cv=StratifiedKFold(5), scoring='neg_mean_squared_error').fit(X_train, y_train)
            selected_features = X_train.columns[rfe.support_]
            X_train_reduced = X_train[selected_features]
            X_test_reduced = X_test[selected_features]
            model.fit(X_train_reduced, y_train)
            y_pred = model.predict(X_test_reduced)
       
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        print(name + " done")

        results[name] = {'MAE': mae, 'MSE': mse, 'selected_feature': selected_features}

            
    return results

# Train and evaluate
# results = evaluate_models(models, X_t, y_t, X_v, y_v)

# # Display Results
# for model_name, metrics in results.items():
#     print("\n")
#     print(f"Model: {model_name}")
#     for metric_name, value in metrics.items():
#         print(f"{metric_name}: {value}")
#     print("\n")

results_feature = feature_select_models(models, X_t, y_t, X_te, y_te)
for model_name, metrics in results_feature.items():
    print("\n")
    print(f"Model: {model_name}")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value}")
    print("\n")

C:\Users\LinBoH\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Linear Regression done


C:\Users\LinBoH\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Lasso done


C:\Users\LinBoH\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


...

Random Forest done


C:\Users\LinBoH\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Gradient Boosting done


C:\Users\LinBoH\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Artificial Neural Network done


Model: Linear Regression
MAE: 19.145577239419847
MSE: 846.8004311931436
selected_feature: Index(['link_length', 'link_freespeed', 'link_capacity', 'link_permlanes',
       'start_node_x', 'start_node_y', 'end_node_x', 'end_node_y',
       'start_count', 'end_count', 'go_to_sum'],
      dtype='object')




Model: Lasso
MAE: 19.556427358308127
MSE: 867.8803264170302
selected_feature: Index(['link_freespeed', 'link_capacity', 'link_permlanes', 'start_node_y',
       'start_count', 'end_count'],
      dtype='object')




Model: Ridge
MAE: 19.61008037407602
MSE: 870.7221155998428
selected_feature: Index(['link_freespeed', 'link_capacity', 'link_permlanes', 'start_node_y',
       'end_node_y', 'start_count', 'end_count'],
      dtype='object')




Model: SVR
MAE: 22.026074260759366
MSE: 1017.7208956151288
selected_feature: Index(['link_id', 'link_to', 'link_length', 'link_freespeed', 'link_capacity',
       'link_permlanes', 'start_node_x', 'start_node_y', 'e

In [54]:
import pickle
with open('result_after_featureselection.pkl', 'wb') as file:
    pickle.dump(results_feature, file)

In [44]:
results


{'Linear Regression': {'MAE': 19.000541966353587, 'MSE': 1350.56069460534},
 'Lasso': {'MAE': 22.610703312854127, 'MSE': 1679.4760262080995},
 'Ridge': {'MAE': 18.989592790164192, 'MSE': 1349.7480663335148},
 'SVR': {'MAE': 23.793351948629688, 'MSE': 1756.1078902638878},
 'Random Forest': {'MAE': 17.635830260141653, 'MSE': 2015.2344103327919},
 'Gradient Boosting': {'MAE': 15.969274342418169, 'MSE': 1596.9989870917648},
 'Artificial Neural Network': {'MAE': 21.345942437519778,
  'MSE': 1888.5103715269568}}

In [45]:
results_feature

{'Linear Regression': {'MAE': 19.47148762371669,
  'MSE': 874.7590684153923,
  'selected_feature': Index(['link_length', 'link_freespeed', 'link_capacity', 'link_permlanes',
         'start_node_x', 'start_node_y', 'end_node_x', 'end_node_y',
         'start_count'],
        dtype='object')},
 'Lasso': {'MAE': 21.29080224423866,
  'MSE': 995.8290100956449,
  'selected_feature': Index(['link_id', 'link_permlanes', 'start_node_x', 'start_node_y',
         'end_node_x', 'end_node_y', 'start_count', 'end_count', 'go_to_sum'],
        dtype='object')},
 'Ridge': {'MAE': 18.855381894067843,
  'MSE': 831.7957770846882,
  'selected_feature': Index(['link_length', 'link_freespeed', 'link_capacity', 'link_permlanes',
         'start_node_x', 'start_node_y', 'end_node_y', 'start_count',
         'end_count'],
        dtype='object')},
 'SVR': {'MAE': 21.113999718501606,
  'MSE': 998.1921690104024,
  'selected_feature': Index(['link_id', 'link_length', 'link_freespeed', 'link_capacity',
         '

In [8]:
param_grid_svr = {
    'C': reciprocal(1e-4, 1e3),  # Extended range for the regularization parameter
    'gamma': reciprocal(1e-4, 1e2),  # Including specific gamma values
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Focusing on RBF kernel
    'epsilon': [0.01, 0.1, 0.2],  # Epsilon in the epsilon-SVR model
}

random_search_svr = RandomizedSearchCV(SVR(max_iter=2000), param_grid_svr, n_iter=70, cv=3, n_jobs=-1, verbose=10, scoring='neg_mean_squared_error')
random_search_svr.fit(X_t, y_t)
print(random_search_svr.best_params_)
print(random_search_svr.best_estimator_)

Fitting 3 folds for each of 70 candidates, totalling 210 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [-2.06753800e+21 -1.78347029e+03 -3.21274664e+21 -4.52036712e+58
 -2.11726699e+03 -1.78346984e+03 -1.78347029e+03 -1.78347029e+03
             nan             nan -6.24658420e+19 -6.10801167e+12
 -1.96408398e+16 -1.78347029e+03             nan -1.78347029e+03
 -1.78347029e+03 -5.64219848e+20             nan -5.87544447e+21
             nan -1.78347029e+03 -1.78347029e+03             nan
 -1.78347029e+03             nan -4.64578815e+15 -1.77140788e+03
             nan -1.78347029e+03 -1.18411241e+16 -1.92220132e+03
 -1.78347029e+03 -1.78346995e+03             nan -2.11002684e+03
 -1.07400560e+17             nan             nan -1.78347029e+03
 -2.04274595e+03 -1.78347029e+03 -1.78347028e+03 -1.78347029e+03
 -1.78347029e+03 -1.78346908e+03 -1.77144765e+03 -1.78204420e+03
 -1.78347029e+03 -1.78347029e+03 -1.78347029e+03 -1.78070475e+03
 -2.11794703e

{'C': 25.383309585489613, 'epsilon': 0.01, 'gamma': 2.1969677491639246, 'kernel': 'rbf'}
SVR(C=25.383309585489613, epsilon=0.01, gamma=2.1969677491639246, max_iter=2000)


C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [4]:
param_grid_rf = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2,4],
    'min_samples_leaf': [1, 2],
    'criterion':['friedman_mse']
}

grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_grid_rf, cv=3, n_jobs=-1, verbose=10, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_t, y_t)
print(grid_search_rf.best_params_)
print(grid_search_rf.best_estimator_)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
{'criterion': 'friedman_mse', 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 150}
RandomForestRegressor(criterion='friedman_mse', max_depth=20,
                      min_samples_leaf=2, n_estimators=150, random_state=42)


In [5]:
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],  # Varied learning rates for gradient boosting
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'subsample': [0.8, 1.0],  # Fraction of samples to be used for fitting individual base learners
}

grid_search_gb = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid_gb, cv=3, n_jobs=-1, verbose=5, scoring='neg_mean_squared_error')
grid_search_gb.fit(X_t, y_t)
print(grid_search_gb.best_params_)
print(grid_search_gb.best_estimator_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
{'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100, 'subsample': 0.8}
GradientBoostingRegressor(max_depth=5, min_samples_split=5, random_state=42,
                          subsample=0.8)


In [6]:
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001, 0.01, 0.1],
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': [0.001, 0.01]
}

grid_search_mlp = GridSearchCV(MLPRegressor(max_iter=1000, random_state=42), param_grid_mlp, cv=3, n_jobs=-1, verbose=5, scoring='neg_mean_squared_error')
grid_search_mlp.fit(X_t, y_t)
print(grid_search_mlp.best_params_)
print(grid_search_mlp.best_estimator_)

Fitting 3 folds for each of 192 candidates, totalling 576 fits
{'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'solver': 'sgd'}
MLPRegressor(activation='tanh', alpha=0.001, learning_rate_init=0.01,
             max_iter=1000, random_state=42, solver='sgd')


In [7]:
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, DotProduct
import numpy as np
param_grid = [{
    "alpha":  [1e-2, 1e-3],
    "kernel": [RBF(l) for l in np.logspace(-1, 1, 2)]
}, {
    "alpha":  [1e-2, 1e-3],
    "kernel": [DotProduct(sigma_0) for sigma_0 in np.logspace(-1, 1, 2)]
}]

gpr = GaussianProcessRegressor()

# Initialize GridSearchCV
grid_search_gpr = RandomizedSearchCV(gpr, param_grid, n_iter=5, cv=3, scoring='neg_mean_squared_error', n_jobs=3, verbose=10)
grid_search_gpr.fit(X_t, y_t)

print(grid_search_gpr.best_params_)
print(grid_search_gpr.best_estimator_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
9 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py", line 310, in fit
    self.L_ = cholesky(K, lower=GPR_CHOLESKY_LOWER, check_finite=False)
  File "C:\Users\user\anaconda3\lib\site-packages\scipy\linalg\_decomp_cholesky.py", line 88, in cholesky
    c, lower = _cholesky(a, lower=

{'kernel': RBF(length_scale=10), 'alpha': 0.01}
GaussianProcessRegressor(alpha=0.01, kernel=RBF(length_scale=10))
